# Arbitrary Precision Operations

How can a computer of finite capability support arbitrary precision operations?

Using dynamic memory allocation. This would make the operations slow.

Don't use arbitrary precision numbers if you can.

Example from lecture notes: Largest exponential.


In [3]:
# import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools    # measure run time

base_exp = [519432 525806; 632382 518061; 78864 613712; 466580 530130; 780495 510032]

bases = base_exp[:,1]
exponents = base_exp[:,2]

# measure runtime
@btime begin
    max_value, max_idx = findmax(BigInt.(bases) .^ BigInt.(exponents))
end

# print value
max_value, max_idx = findmax(BigInt.(bases) .^ BigInt.(exponents))
println("Bigint: Largest value on row ", max_idx, ": ", bases[max_idx], "^", exponents[max_idx])

# measure runtime
@btime begin
    max_value, max_idx = findmax(exponents .* log.(bases))
end

# print value
max_value, max_idx = findmax(exponents .* log.(bases))
println("Log: Largest value on row ", max_idx, ": ", bases[max_idx], "^", exponents[max_idx])

# finite digit operation is much better in both time and memory
# (1 ms = 1000,000 ns)

  185.920 ms (1213 allocations: 52.11 MiB)
Bigint: Largest value on row 5: 780495^510032
  869.808 ns (6 allocations: 256 bytes)
Log: Largest value on row 5: 780495^510032


## Ways to loop (continue)

- `for` or `while` loop applied to each element.  
 **prons**: flexible; **cons**: less efficient.

- Vectorized operations.  
 **prons**: efficient, better readability in some cases; **cons**: less flexible;

- Recursion.  
 **prons**: logically easy for some tasks; **cons**: inefficient, hard to trace.

In [4]:
using Random            # sample random numbers
using BenchmarkTools    # measure run time

Random.seed!(1)         # for reproducibility

# a vector of 100 random numbers
x = rand(100) * 10

# compare running time of different ways doing the same task

# elementwise loop
@btime begin
    diff_prod = 1
    for i = 1:(length(x)-1)
        diff_prod *= x[i+1] - x[i]
    end
end

# vectorized operation
@btime begin
    diff_prod = prod(x[2:end] - x[1:end-1])
end

# recursion
function calc_diff_prod(x)
    if length(x) == 2
        return (x[2] - x[1])
    end
    
    return (x[end] - x[end-1]) * calc_diff_prod(x[1:end-1])
end

@btime begin
    diff_prod = calc_diff_prod(x)
end

# vectorized operation is better in both time and memory
# (1 us = 1000 ns)

  15.515 μs (496 allocations: 9.31 KiB)
  434.950 ns (6 allocations: 2.70 KiB)
  10.289 μs (99 allocations: 48.11 KiB)


-1.6005666065789959e34